In [ ]:
import numpy as np
import math

CA = np.loadtxt("CA.txt") # All GPS satellites' C/A data
G13 = np.loadtxt("G13.txt") # 10 epoch X, Y, Z and clock error included
G14 = np.loadtxt("G14.txt") # 10 epoch X, Y, Z and clock error included
G15 = np.loadtxt("G15.txt") # 10 epoch X, Y, Z and clock error included
G17 = np.loadtxt("G17.txt") # 10 epoch X, Y, Z and clock error included
G19 = np.loadtxt("G19.txt") # 10 epoch X, Y, Z and clock error included
G30 = np.loadtxt("G30.txt") # 10 epoch X, Y, Z and clock error included

G13_CA = CA[0][1]
G14_CA = CA[1][1]
G15_CA = CA[2][1]
G17_CA = CA[3][1]
G19_CA = CA[4][1]
G30_CA = CA[5][1]

approximate_position = np.loadtxt("approximate_position.txt")

#TGD from broadcast ephemeris data
TGD_s = np.loadtxt("TGD.txt") #second

#Temmis clock errors
clock_errors = np.loadtxt("clock_errors.txt")

# ION Data from brd file 
ions = np.loadtxt("ions.txt")
ion_alpha = ions[0]
ion_beta = ions[1]
twe_ion  = 28500 + 3 * 24 * 60 * 60 # Seocnds of week for Ion_Kloubchar

# Final position computaion result of satellite from Assignment 2
sat_pos_G13 = [-1609411.036954920,  11298978.11054770, -2803663.98620605]
sat_pos_G14 = [ 4083974.100353130, -12074992.79641555,  106866.455078120]
sat_pos_G15 = [ 665838.0477212200,  20205047.20543393,  132331.719970700]
sat_pos_G17 = [ 13544619.41542759, -3647536.154782970,  4852910.81542969]
sat_pos_G19 = [-423111.4911553000,  5689109.507641980,  1531721.98486328]
sat_pos_G30 = [-11268939.42513021,  494918.0615515800, -363019.906616210]

# Constants
c = 299792458 # Velocity of light
u = 3.986005e14 # Earth's gravitational constant (WGS84)
we = 7.2921151467e-05 # Earth's rotation rate (WGS84)

def xyz2plh(cart):
    
    # WGS84 ellipsoid parameters
    a = 6378137.0  # semi-major axis
    f = 1 / 298.257223563  # flattening

    b = a * (1 - f)  # semi-minor axis
    e_sq = (a**2 - b**2) / a**2  # eccentricity squared
    
    x, y, z = cart
    p = math.sqrt(x**2 + y**2)  # distance from Z-axis

    # Initial guess for latitude
    theta = math.atan2(z * a, p * b)

    # Iterative calculation for latitude
    delta = 1.0
    while delta > 1e-12:
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)

        N = a / math.sqrt(1 - e_sq * sin_theta**2)
        h = p / cos_theta - N

        prev_theta = theta
        theta = math.atan2(z + N * e_sq * sin_theta, p)

        delta = abs(theta - prev_theta)

    # Calculate latitude, longitude, and height
    latitude = math.degrees(theta)
    longitude = math.degrees(math.atan2(y, x))
    height = h

    return latitude, longitude, height


def local(rec, sat):
    
    # WGS84 ellipsoid parameters
    a = 6378137.0  # semi-major axis
    f = 1 / 298.257223563  # flattening

    rx, ry, rz = rec # Receiver coordinates
    sx, sy, sz = sat # Satellite coordinates

    # Computes equatıon
    dx = sx - rx
    dy = sy - ry
    dz = sz - rz

    # Computing slant distance
    slantd = math.sqrt(dx * dx + dy * dy + dz * dz)/100 # km

    # Convert azimuth and zenith angles
    azimuth = math.atan2(dy, dx)
    zenith = math.atan2(dz, math.sqrt(dx * dx + dy * dy))

    # Convert radians to degrees
    azimuth = math.degrees(azimuth) # degree
    zenith = math.degrees(zenith) # degree
    
    return azimuth, zenith, slantd

# azG13, zenG13, slantdG13 = local(approximate_position, sat_pos_G13)
# azG14, zenG14, slantdG14 = local(approximate_position, sat_pos_G14)
# azG15, zenG15 , slantdG15 = local(approximate_position, sat_pos_G15)
# azG17, zenG17 , slantdG17 = local(approximate_position, sat_pos_G17)
# azG19, zenG19 , slantdG19 = local(approximate_position, sat_pos_G19)
# azG30, zenG30 , slantdG30 = local(approximate_position, sat_pos_G30)

elev_G13= 90 - (-32.7203087526703)
elev_G14= 90 - (-13.788445099679372)
elev_G15= 90 - (-11.652123948903526)
elev_G17= 90 - (  5.395974424357155)
elev_G19= 90 - (-22.44620668186342)
elev_G30= 90 - (-14.786434690653898)



                                        # ION COMPUTAIONS

"""
    The tropospheric model developed by[Collins, 1999] and adopted by the
    Satellite - Based Augmentation System(SBAS).Details of the model are
    presented in: GNSS Data Processing.J.Sanz Subirana, J.M.Juan Zornoza
    and M.Hernández - Pajares, 2013, Vol.1, (ESA TM-23 / 1, May 2013),pp.123 - 124
    
    Inputs
    lat: Latitude of the receiver[deg]
    D: Day of year[integer]
    H: Height of the station above mean sea level[m]
    E: Elevation of the signal[rad]
    Outputs
    Trzd: Zenith dry(hydrostatic) delay[m]
    Trzw: Zenith wet delay[m]
    ME: Mapping function for both dry and wet delays
"""


def trop_SPP(lat, D, H, E):
     # Average meteorological parameters for the tropospheric delay
     # pressure [P(mbar)], temperature [T(K)],
     # water vapour pressure [e(mbar)], temperature rate [Beta(K/m)] and  
     # water vapour rate [Alpha (Dimensionless)]
    Met15 = [1013.25, 299.65, 26.31, 6.30e-3, 2.77]
    Met30 = [1017.25, 294.15, 21.79, 6.05e-3, 3.15]
    Met45 = [1015.75, 283.15, 11.66, 5.58e-3, 2.57]
    Met60 = [1011.75, 272.15, 6.78, 5.39e-3, 1.81]
    Met75 = [1013.00, 263.65, 4.11, 4.53e-3, 1.55]

    # Seasonal variations for the meteorological parameters
    dMet15 = [0.00, 0.00, 0.00, 0.00e-3, 0.00]
    dMet30 = [-3.75, 7.00, 8.85, 0.25e-3, 0.33]
    dMet45 = [-2.25, 11.00, 7.24, 0.32e-3, 0.46]
    dMet60 = [-1.75, 15.00, 5.36, 0.81e-3, 0.74]
    dMet75 = [-0.50, 14.50, 3.39, 0.62e-3, 0.30]

    A = [0] * 5
    B = [0] * 5

    if lat <= 15:
        A = Met15
        B = dMet15
    elif lat > 15 and lat <= 30:
        for i in range(5):
            A[i] = Met15[i] + ((lat - 15) / 15) * (Met30[i] - Met15[i])
            B[i] = dMet15[i] + ((lat - 15) / 15) * (dMet30[i] - dMet15[i])
    elif lat > 30 and lat <= 45:
        for i in range(5):
            A[i] = Met30[i] + ((lat - 30) / 15) * (Met45[i] - Met30[i])
            B[i] = dMet30[i] + ((lat - 30) / 15) * (dMet45[i] - dMet30[i])
    elif lat > 45 and lat <= 60:
        for i in range(5):
            A[i] = Met45[i] + ((lat - 45) / 15) * (Met60[i] - Met45[i])
            B[i] = dMet45[i] + ((lat - 45) / 15) * (dMet60[i] - dMet45[i])
    elif lat > 60 and lat < 75:
        for i in range(5):
            A[i] = Met60[i] + ((lat - 60) / 15) * (Met75[i] - Met60[i])
            B[i] = dMet60[i] + ((lat - 60) / 15) * (dMet75[i] - dMet60[i])
    elif lat >= 75:
            A = Met75
            B = dMet75

    ME = 1.001 / math.sqrt(0.002001 + math.sin(E) ** 2)

    k1 = 77.604
    k2 = 382000
    Rd = 287.054
    gm = 9.784
    g = 9.80665

    if lat > 0:
        Dmin = 28
    else:
        Dmin = 211

    P = A[0] + B[0] * math.cos((2 * math.pi * (D - Dmin)) / 365.25)
    T = A[1] + B[1] * math.cos((2 * math.pi * (D - Dmin)) / 365.25)
    e = A[2] + B[2] * math.cos((2 * math.pi * (D - Dmin)) / 365.25)
    Beta = A[3] + B[3] * math.cos((2 * math.pi * (D - Dmin)) / 365.25)
    Alpha = A[4] + B[4] * math.cos((2 * math.pi * (D - Dmin)) / 365.25)

    Trz0d = (1e-6 * k1 * Rd * P) / gm
    Trz0w = ((1e-6 * k2 * Rd) / (((Alpha + 1) * gm) - Beta * Rd)) * (e / T)

    Trzd = ((1 - ((Beta * H) / T)) ** (g / (Rd * Beta))) * Trz0d
    Trzw = ((1 - ((Beta * H) / T)) ** ((((Alpha + 1) * g) / (Rd * Beta)) - 1)) * Trz0w

    return Trzd, Trzw


    """
    This function compute the ionospheric slant delay for GPS code measeurement in L1 signal
    Inputs
    lat  : geodetic latitude (radian) from approximate coordinates in the observation file
    lon  : geodetic longitude(radian) from approximate coordinates in the observation file
    elv  : elevation angle   (radian) for the related satellite
    azm  : azimuth angle     (radian) for the related satellite
    alfa : Klobuchar coefficients (sec sec/semicircle sec/semicircle2 sec/semicircle3) 
    beta : Klobuchar coefficients (sec sec/semicircle sec/semicircle2 sec/semicircle3)
    tgps : gps time (seconds of week) for the observation
    Output
    dion : ionospheric slant delay (meter) for the observation
    Reference
    GNSS Data Processing, Vol. I: Fundamentals and Algorithms (ESA TM-23/1, May 2013)
    """

def Ion_Klobuchar(lat, lon, elv, azm, alfa, beta, tgps):
    # velocity of light
    c = 299792458  # m/s

    # calculate the Earth-centred angle
    Re = 6378  # km
    h = 350  # km
    cns = (Re / (Re + h)) * math.cos(elv)
    eca = math.pi / 2 - elv - math.asin(cns)

    # compute the latitude of IPP
    ax = math.sin(lat) * math.cos(eca) + math.cos(lat) * math.sin(eca) * math.cos(azm)
    lat_ipp = math.asin(ax)

    # compute the longitude of IPP
    lon_ipp = lon + (eca * math.sin(azm)) / (math.cos(lat_ipp))

    # Find the geomagnetic latitude of the IPP
    f_pol = math.radians(78.3)
    l_pol = math.radians(291)
    
    as_ = math.sin(lat_ipp) * math.sin(f_pol) + math.cos(lat_ipp) * math.cos(f_pol) * math.cos(
        lon_ipp - l_pol)
    
    geo = math.asin(as_)

    # Find the local time at the IPP
    t = 43200 * (lon_ipp / math.pi) + tgps
    t = t % 86400  # Seconds of day
    if t >= 86400:
        t = t - 86400
    elif t <= 0:
        t = t + 86400

    tsd = geo / math.pi
    AI = alfa[0] + alfa[1] * tsd + alfa[2] * (tsd ** 2) + alfa[3] * (tsd ** 3)  # seconds
    PI = beta[0] + beta[1] * tsd + beta[2] * (tsd ** 2) + beta[3] * (tsd ** 3)  # seconds
    if AI < 0:
        AI = 0
    if PI < 72000:
        PI = 72000

    # Compute the phase of ionospheric delay
    XI = (2 * math.pi * (t - 50400)) / PI  # radian

    # Compute the slant factor (ionospheric mapping function)
    F = (1 - cns ** 2) ** (-1 / 2)

    # Compute the ionospheric time delay
    if abs(XI) < (math.pi / 2):
        I1 = (5 * (10 ** (-9)) + AI * math.cos(XI)) * F
    elif abs(XI) >= (math.pi / 2):
        I1 = (5 * (10 ** (-9))) * F

    dion = I1 * c
        
    return dion



def atmos_G13(doy, trec, trecw, C1, rec, sp3, alpha, beta):
    ellipsoid = xyz2plh(approximate_position)
    latitude, longitude, height = xyz2plh(approximate_position)
    azG13, zenG13, slantdG13 = local(approximate_position, sat_pos_G13)
    TrD, TrW = trop_SPP(ellipsoid[0], doy, ellipsoid[2], trecw)
    IonD_G13 = Ion_Klobuchar(ellipsoid[0], ellipsoid[1], elev_G13, azG13, ion_alpha, ion_beta, twe_ion)
    
    '''
    print(" "*30, "-"*53)
    print(" "*30, "|"," "*4,"Ion Delay G13     = ", IonD_G13, " "*5, "|")
    '''
    
def atmos_G14(doy, trec, trecw, C1, rec, sp3, alpha, beta):
    ellipsoid = xyz2plh(approximate_position)
    latitude, longitude, height = xyz2plh(approximate_position)
    azG14, zenG14, slantdG14 = local(approximate_position, sat_pos_G14)
    TrD, TrW = trop_SPP(ellipsoid[0], doy, ellipsoid[2], trecw)
    IonD_G14 = Ion_Klobuchar(ellipsoid[0], ellipsoid[1], elev_G14, azG14, ion_alpha, ion_beta, twe_ion)
    
    '''
    print(" "*30, "-"*53)
    print(" "*30, "|"," "*4,"Ion Delay G14     = ", IonD_G14, " "*5, "|")
    '''
    
def atmos_G15(doy, trec, trecw, C1, rec, sp3, alpha, beta):
    ellipsoid = xyz2plh(approximate_position)
    latitude, longitude, height = xyz2plh(approximate_position)
    azG15, zenG15 , slantdG15 = local(approximate_position, sat_pos_G15)
    TrD, TrW = trop_SPP(ellipsoid[0], doy, ellipsoid[2], trecw)
    IonD_G15 = Ion_Klobuchar(ellipsoid[0], ellipsoid[1], elev_G15, azG15, ion_alpha, ion_beta, twe_ion)
    
    '''
    print(" "*30, "-"*53)
    print(" "*30, "|"," "*4,"Ion Delay G15     = ", IonD_G15, " "*4, "|")
    '''
    
def atmos_G17(doy, trec, trecw, C1, rec, sp3, alpha, beta):
    ellipsoid = xyz2plh(approximate_position)
    latitude, longitude, height = xyz2plh(approximate_position)
    azG17, zenG17 , slantdG17 = local(approximate_position, sat_pos_G17)
    TrD, TrW = trop_SPP(ellipsoid[0], doy, ellipsoid[2], trecw)
    IonD_G17 = Ion_Klobuchar(ellipsoid[0], ellipsoid[1], elev_G17, azG17, ion_alpha, ion_beta, twe_ion)
    
    '''
    print(" "*30, "-"*53)
    print(" "*30, "|"," "*4,"Ion Delay G17     = ", IonD_G17, " "*4, "|")
    '''
    
def atmos_G19(doy, trec, trecw, C1, rec, sp3, alpha, beta):
    ellipsoid = xyz2plh(approximate_position)
    latitude, longitude, height = xyz2plh(approximate_position)
    azG19, zenG19 , slantdG19 = local(approximate_position, sat_pos_G19)
    TrD, TrW = trop_SPP(ellipsoid[0], doy, ellipsoid[2], trecw)
    IonD_G19 = Ion_Klobuchar(ellipsoid[0], ellipsoid[1], elev_G19, azG19, ion_alpha, ion_beta, twe_ion)
    
    '''
    print(" "*30, "-"*53)
    print(" "*30, "|"," "*4,"Ion Delay G19     = ", IonD_G19, " "*5, "|")
    '''
    
def atmos_G30(doy, trec, trecw, C1, rec, sp3, alpha, beta):
    ellipsoid = xyz2plh(approximate_position)
    latitude, longitude, height = xyz2plh(approximate_position)
    azG30, zenG30 , slantdG30 = local(approximate_position, sat_pos_G30)
    TrD, TrW = trop_SPP(ellipsoid[0], doy, ellipsoid[2], trecw)
    IonD_G30 = Ion_Klobuchar(ellipsoid[0], ellipsoid[1], elev_G30, azG30, ion_alpha, ion_beta, twe_ion)
    
    '''
    print(" "*30, "-"*53)
    print(" "*30, "|"," "*4,"Ion Delay G30     = ", IonD_G30, " "*5, "|")
    print(" "*30, "-"*53)
    print(" "*30, "|", " "*4, "Troposphere Dry   = ", TrD, " "*4, "|")
    print(" "*30, "|"," "*4,"Troposphere Wet   = ", TrW, " "*3, "|")
    print(" "*30, "-"*53)
    '''
    
atmos_G13(60, 28500, twe_ion, G13_CA, approximate_position, sat_pos_G13, ion_alpha, ion_beta)
atmos_G14(60, 28500, twe_ion, G14_CA, approximate_position, sat_pos_G14, ion_alpha, ion_beta)
atmos_G15(60, 28500, twe_ion, G15_CA, approximate_position, sat_pos_G15, ion_alpha, ion_beta)
atmos_G17(60, 28500, twe_ion, G17_CA, approximate_position, sat_pos_G17, ion_alpha, ion_beta)
atmos_G19(60, 28500, twe_ion, G19_CA, approximate_position, sat_pos_G19, ion_alpha, ion_beta)
atmos_G30(60, 28500, twe_ion, G30_CA, approximate_position, sat_pos_G30, ion_alpha, ion_beta)


#                                                OUTPUTS OF ION DELAY
#
#                               -----------------------------------------------------
#                               |      Ion Delay G13     =  4.061982364760598       |
#                               -----------------------------------------------------
#                               |      Ion Delay G14     =  4.452222197716992       |
#                               -----------------------------------------------------
#                               |      Ion Delay G15     =  1.6450943567929928      |
#                               -----------------------------------------------------
#                               |      Ion Delay G17     =  3.9536339384646038      |
#                               -----------------------------------------------------
#                               |      Ion Delay G19     =  8.409797670871829       |
#                               -----------------------------------------------------
#                               |      Ion Delay G30     =  5.855484944510037       |
#                               -----------------------------------------------------
#                               |      Troposphere Dry   =  2.2993992000775143      |
#                               |      Troposphere Wet   =  0.22305722376610435     |
#                               -----------------------------------------------------

IonD = [4.061982364760598,
       4.452222197716992,
       1.6450943567929928,
       3.9536339384646038,
       8.409797670871829,
       5.855484944510037,]

TrD = 2.2993992000775143
TrW = 0.22305722376610435

TGD_computed = c*TGD_s # multiply by speed of light.

#Compute all D values.
D_matrix = np.zeros((5,1))

for i in range(5):
    D = (-c*clock_errors[i]*10e-6) + TrD + IonD[i] + TGD_computed[i]
    D_matrix[i] = D

def pj(xj, yj, zj, x0, y0, z0):
    return np.sqrt((xj - x0)*2 + (yj - y0)*2 + (zj - z0)**2)

for i in range(5):
    AT = np.transpose(A)
    ATA = np.dot(AT, A)
    ATA_inverse = np.linalg.inv(ATA)
    ATl = np.dot(AT, l)

    x_matrix = np.dot(ATA_inverse, ATl)
    print(x_matrix)

x0 = 0.0
y0 = 0.0
z0 = 0.0


rec_cl_offset= np.zeros((5,1))

for i in range(0,5):
    c_er = (R[i] - D_matrix[i] - lin_appr[i])
    rec_cl_offset[i] = c_er/c


dx = (xj - x0)
dy = (yj - y0)
dz = (zj - z0)

l = np.array([R[i] - lin_appr[i] - D_matrix[i]])
A = np.array([(x0-xj[i])/lin_appr[i]], [(y0-yj[i])/lin_appr[i]], [(z0-zj[i])/lin_appr[i]], [1])

x_matrix = np.array([dx, dy, dz, rec_cl_offset[i]])


?
?
?
?
?

'''

Günlerce uğraştım sonuçsuz kaldı...
:( :( :(

'''
